In [3]:
from langchain.llms.gpt4all import GPT4All
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "You are a helpful assistant that defines words. Define this word: {word}"
)

llm = GPT4All(model="./mistral.bin")

chain = prompt | llm

chain.invoke({"word": "potato"})

'\n\nPotato (noun) - A starchy vegetable that is grown underground in rows, and has a round or oblong shape with a smooth skin that can be various colors such as white, yellow, red, or purple. Potatoes are commonly used as a source of carbohydrates and can be boiled, baked, fried, roasted, mashed or processed into flour. They originated in South America but are now grown worldwide.'